# Concurrency Models & Executors


## Design patterns in this notebook and `patterns` folder

These designs are adapted from Chapters 19 and 20 of [Fluent Python](https://github.com/fluentpython/example-code-2e).

- A (main) supervisor main process/thread spins a secondary process/thread to execute a concurrent job (e.g. to show progress update). The design is explained in the examples below and in:
    - [patterns/spinner_proc.py](patterns/spinner_proc.py)
    - [patterns/spinner_async.py](patterns/spinner_async.py)
    - see examples below for multi-thread implementation

- Jobs are sent to a queue and processed by a number of processes/threads. 
    - See instructions into [patterns/queue_procs.py](patterns/queue_procs.py)
    - For a thread based implementation of the above, look into [this example from  fluentpython](https://github.com/fluentpython/example-code-2e/blob/master/19-concurrency/primes/threads.py).

- Jobs are sent to a pool of executors. This is exemplified for both multi-thread and multi-processes in [patterns/pool_thread_and_proc.py](patterns/pool_thread_and_proc.py) using `concurrent.futures.ProcessPoolExecutor` and `concurrent.futures.ThreadPoolExecutor`. The script showcases two `main` functions.
    - `main_no_error_handling`: simplier implementation using the `map`. It automatically preserves results order, but does not allow error handling (at least not outside the task function);
    - `main_error_handling`: A slightly more complex one, using th `submit` and `as_completed` methods, which allows executing different tasks, error handling and returning the results as soon as ready. This second pattern uses [Futures](#Futures).
 

# Concurrency Models

There is confusion about terminology between concurrency and parallelism.

> Concurrency is about dealing with lots of things at once.

> Parallelism is about doing lots of things at once.

> Not the same, but related.

> One is about structure, one is about execution.

> Concurrency provides a way to structure a solution to solve a problem that may (but not necessarily) be parallelizable.

> *—Rob Pike, co-inventor of the Go language1*

In Pike’s view, **all parallel systems are concurrent, but not all concurrent systems are parallel.**


# Jargon

- **Concurrency**: ability to handle multiple tasks, making progress one at a time or in parallel. 

    - **Two ways to achieve it**:
        - **(Preemptive) Multitasking**: a single-core CPU is capable of concurrency if an **OS scheduler** interleaves the execution of the pending tasks. 
            - Namely, the OS temporarely interrupts an executing task, to execute something else, and with the intention to resume eecution later (until error or complition).
            - There is also **cooperative multi-tasking** (see below), but this is within the same process. 

        - **Parallelism** The ability to execute multiple computations at the same time. (multicore CPU, multiple CPUs, a GPU, or multiple computers in a cluster).

    - **Execution unit**:
        - objects that execute code concurrently (either in parallel or multitasking), each with **independent state** and **call stack**. 
        - Python supports three kinds of execution units: processes, threads, and coroutines.


- **Process**. an **instance** of a computer program **while is running**.
    - **While is running** means that the process **uses memory** and **is allocated a slide of CPU**, namely some time on CPU.
    - **OS system** manages processes concurrently.
        - Each process has **its own slice of private memory**
        - Processes can communicate through pipes, sockets or memory mapped files.
            - **Python objects must be serialised (i.e. converted into byte) to be shared with other (not necessarily Python) processes**.
            - **Due to seriailisation, not all Python objects can be shared across processes**.
    - A process can create subprocesses, each being isolated (i.e. having its own memory)
    - In Python, creating multiple **processes** allows preemptive multi-tasking or parallelism.

- **Thread**. An execution unit with a unique process. 
    - When a process starts, is always single thread.
    - A process can then create multiple thread - by calling the OS API.
    - Threads within a process **share the same memory**, hence:
        - **Python threads can share any objects**
        - **Data (objects or files) can be corrupted if multiple thread access at the same time**
    - Threads also allow preemptive multitasking.
    - Threads consume less resources than processes (possibly due to memory share)

- **Coroutine** a function that can suspend itself and resume later.
    - two types:
        - classic coroutines are built from generator functions.
        - native coroutines are defined with `async def`
    - Python coroutines **usually** run within a single thread under the supervision of an event loop, also in the same thread. 
        - Asynchronous programming frameworks such as asyncio, Curio, or Trio provide an event loop and supporting libraries for nonblocking, coroutine-based I/O. 
    - Coroutines support **cooperative multitasking**: 
        - each coroutine **explicitly cede control with the yield or await keyword**, so that another may proceed concurrently (but **not in parallel**). 
        - This means that any blocking code in a coroutine blocks the execution of the event loop and all other coroutines. 
            — In contrast with the preemptive multitasking supported by processes and threads (though in this case, the OS scheduler will still de facto interrupt some of the tasks).
        - On the other hand, each coroutine consumes less resources than a thread or process doing the same job


- **Queue** a data structure that let's you put and get items.
    - usually in FIFO (first input first output).
    - Queues allow separate execution units to exchange application data and control messages (error codes, signals to terminate, etc). 
    - The implementation of a queue varies according to the underlying concurrency model. In Python:
        - the `queue` model provides classes to support threads.
        - `multipticessing` and `asyncio` implement their own classes, including queues that are not FIFO: `LifoQueue` and `PriorityQueue`.

- **Lock** an object that execution units can use to sync their actions (and avoid corrupting data).

    - While updating a shared data structure, the running code should hold an associated lock. This signals other parts of the program to wait until the lock is released. 
    - The simplest type of lock is also known as a mutex (for mutual exclusion). 
    The implementation depends on the underlying concurrency model.

- **Contention** Dispute over a limited asset.
    - Happens when multiple execution units try to access a shared resource — such as a lock or storage. 
    - There’s also CPU contention, when compute-intensive processes or threads must wait for the OS scheduler to give them a share of the CPU time.




## How concurrency applies in Python

- An instance of Python is a process (like an instance of any other program, I presume).

    - From Python you can start other processes. Either:
        - Python processes using the `multiprocessing` or `concurrent.futures` libraries. 
        - external pro‐ grams (any language) using the `subprocess` library.

- The Python interpreter uses a single thread. 
    - This runs:
        - the user’s program and 
        - the memory garbage collector. 
    - You can start additional Python threads with `threading` or `concurrent.futures`.

- **GIL**. Within the python interpreter thread (and I presume within the memory garbage collector), lives the **Global Interpreter Lock (GIL)**

    - This controls access to the object reference counts and other internal interpreter states.
    - **Only one Python thread is allowed to hold the GIL at any time**. This means that:
        - only one thread can (say) modify the object reference counts or other states.
        - More specifically, **only one thread can run at a given time within a python process** (regadrless of the number of CPU cores). 
            - Two threads could not run at the same time, cause they would not be able to modify the object reference counts or other internal states.

- To prevent a Python thread from holding the GIL indefinitely, Python’s bytecode interpreter pauses the current Python thread every 5ms, releasing the GIL. 
    - The thread can then try to reacquire the GIL, but if there are other threads waiting for it, the OS scheduler may pick one of them to proceed.
    - Call `sys.getswitchinterval()` to get the interval; 
    - change it with `sys.setswitchinterval(s)`.

-  When we write Python code, **we have no control over the GIL** (meaning we can not alter how the GIL works, nor release it). 
    - **GIL is not part of the Python language definition**.
    - The Jython and IronPython implementations don’t have a GIL. Unfortunately, both are lagging behind

- However, **some built-in function or an extension written in C**, or any language that interfaces at the Python/C API level, **can release the GIL while running time-consuming tasks**
    - Every Python standard library function that makes a *syscall* (i.e. to a function of the OS kernel) releases the GIL. 
        - This includes all functions that perform disk I/O, network I/O, and time.sleep(). 
        - Many CPU-intensive functions in the NumPy/SciPy libraries, as well as the compressing/decompressing functions, also release the GIL.

    - This means that (say when you are writing a large file):
        - If you write your code in single thread, you will need to way until writing is done to go to the next operation (line of code);
        - But if you wrote in multi-thread, once the thread 1 starts writing, it will release the GIL, allowing another thread to continue to do work.
        - Once thread 1 has finished, it will start requesting the GIL again (so that it can go to the next operation)

- Extensions that integrate at the Python/C API level can also launch other non-Python threads that are not affected by the GIL. 
    - Such GIL-free threads generally cannot change Python objects, but they can read from and write to the memory underlying objects that support the buffer protocol, such as bytearray, array.array, and NumPy arrays.

- **Contention and GIL**
    - The effect of the GIL on network programming with Python threads is relatively small. In fact:
        - I/O functions release the GIL, and 
        - reading or writing to the network always implies high latency—compared to reading and writing to memory. 
    - Consequently, each individual thread spends a lot of time waiting anyway, so their execution can be interleaved without major impact on the overall throughput. 

- Contention over the GIL **slows down compute-intensive Python threads**. 
    - This is because compute-intensive threads are *actually doing work* (not waiting for a I/O to finish).
    - hence, they will be continually asking for the GIL to be released.
    - Sequential, single-threaded code is simpler and faster for such tasks.
    - To run CPU-intensive Python code on multiple cores, you must use multiple Python processes.

## Multi-thread example

In this program we have 2 function:
- a slow to run function that releases the GIL.
    - In real world this is a I/O function, or one that makes a system call.
- a function that *keeps using the Python interpreter*, in this example to show progress.


In the main process we create an object, `done: Event`, that holds True/False states.

The function `spin`, which will show progress in terminal, is started in a ** new, secondary, thread**. 

The show progress function `spin` has an object, `done: Event`, 
    - Initially `done is False`, and the `spin` function is written so as to show a spinning wheel until `done is False`.

The main thread will run the `supervisor` function. 
- It first creates and starts the secondary thread.
- It then starts the slow function (in real life a I/O or call to network).

The `done` argument is shared between the two threads: once the main thread has completed the job, it will set it to `True`, signalling to the second thread that it can complete.

**Note:** you can not share any argument between threads. If you replace `done` with a simple `bool`, the code won't work. 

In [3]:
import itertools
import time
from threading import Thread, Event


def spin(msg: str, done: Event) -> None:
    refresh_rate = 0.1

    # Progress function. Will run in thread 1.
    for char in itertools.cycle(r'\|/-'):
        # The `\r` moves cursor back at beginning of the start of the line
        status = f'\r{char} {msg}'
        print(status, end='', flush=True)

        # Check if other thread is ready.
        # - Wait effectively blocks this thread (hence the function) for some time.
        # - The thread is stopped for a maximum of `refresh rate` seconds.
        # - Unless some other thread sets the Event to True in the meanwhile, execution resumes.
        # - the fact that the thread stops for a while is what allows us to see the animation, else
        #   the spin would spin too fast!
        if done.wait(refresh_rate):
            break
    blanks = ' ' * len(status)
    print(f'\r{blanks}\r', end='')


def slow() -> int:
    # A slow to execute function that is releasing the GIL (e.g. writing I/O).
    # Will run in thread 2.
    time.sleep(3)
    return 1


def supervisor() -> int:

    # Create an event to signal when the slow process is completed.
    # - Initially, done is False
    # - Note. This must be an instance of Event. If you replace with a simple bool, the spinner
    #   thread will start, but will never complete.
    done = Event()

    # Create (and start) a different thread to show the progress.
    # - We specify which function is executed in the thread, as well as its arguments.
    # - One of the arguments is the `done` event, that will allow us to signal when the thread must
    #   be closed
    spinner = Thread(target=spin, args=('thinking!', done))
    print(f'spinner object: {spinner}')
    spinner.start()

    # Now we start a slow operation in the current thread.
    # - This operation is formally blocking the main thread.
    # - T spinner thread is already running in the background, instead.
    result = slow()

    # Once result is available, I can change the event status.
    # - The spin function will terminate as soon as it sees `done is True`
    done.set()

    # In fact, the spinner thread does not terminate instantly (as it checks for `done` being True
    # only every x seconds). Hence, with this command I officially tell the main thread to wait for
    # spinner thread to terminate, before continuing.
    spinner.join()

    # Now that spinner is closed, I can terminate the function and return the outcome.
    return result


result = supervisor()
print(f'Answer: {result}')

spinner object: <Thread(Thread-7 (spin), initial)>
Answer: 1   


## Multiprocessing

The API imitates the `threading` API, though there are some differences (see code comments). 

With multiprocessing, each python process has their own GIL, so is suitable for parallel programming and exploiting multiple cores (though the last answer is always of the OS, which orchestrates CPUs usage).

Despite processes not sharing the same memory, we can still share (simple) variables using the `multiprocessing` API; these object written low level in C. 

**IMPORTANT** this code requires to be in a script (with `if __name__=='__main__'`). See [19-concurrency-models/spinner_proc.py](spinner_proc.py).

In [4]:
import os
os.system('python spinner_proc.py')

spinner object: <Process name='Process-1' parent=34254 initial>


0

## Coroutines

For threads and processes, it is the OS schedulers that allocates CPU time.

Instead, coroutines are driven by an **application-level event loop** (created with `asyncio.run`). This event loop manages a queue of pending coroutines. 
- When some of these coroutines trigger an I/O operation, the event loop passes control to other coroutines in the event loop; 
- when the I/O is over, the event loop passes control back to the original coroutine.


**Note that the event loop and the library coroutines and the user coroutines all execute in a single thread**. Therefore:
- any time spent in a coroutine pauses the event loop, and all other coroutines;
- as it is a single thread, there is no mechanism of releasing the GIL. Hence, if I make a system call (e.g. `time.sleep()`, I am going to block the whole program)


See [19-concurrency-models/spinner_proc.py](spinner_async.py) for an example.

The functioning of coroutines is based on (below `coro()` indicates a call to a coroutine function, hence a function defined as `async def coro():`):

- `asyncio.run(coro())`. Called from a regular function to drive a coroutine object that usually is the entry point for all the asynchronous code in the program, like the supervisor in this example. This call creates the event loop and blocks until the body of coro returns. The return value of the run() call is whatever the body of coro returns.

- `asyncio.create_task(coro())` Called from a coroutine to schedule another coroutine to execute. **This call does not suspend the current coroutine**. It returns a Task instance, an object that wraps the coroutine object and provides methods to control and query its state.
    - For example this can be used to start a coroutine that shows progress update, and that will be killed once the calling coroutine has terminated its task.

- `await coro()`. Called from a coroutine to transfer control to the coroutine object returned by coro(). **This suspends the current coroutine until the body of coro returns**. The value of the await expression is whatever the body of coro returns.

## Futures

There are two classes named `Future` in the standard library: `concurrent.futures.Future` and `asyncio.Future`. An instance of either class represents a deferred computation that may or may not have completed. This is somewhat similar to the Deferred class in Twisted, the Future class in Tornado, and Promise in modern JavaScript.

Futures encapsulate pending operations so that we can put them in queues, check whether they are done, and retrieve results (or exceptions) when they become available.

They are meant to be instantiated exclusively by the concurrency framework, be it `concurrent.futures` or `asyncio` and **NOT BY THE USER**. This is because they represent something that will eventually run, therefore it must be scheduled to run, which is the job of the framework. 
- In particular, `concurrent.futures.Future` instances are created by submitting a callable for execution with a `concurrent.futures.Executor` subclass (`Executor.submit()`).

Application code is not supposed to change the state of a future.

Futures have a `.done()` method, though normally, instead of repeatedly asking whether a future is done, client code usually asks to be notified through `.add_done_call back()` method.


There is also a `.result()` method, which returns the result of the callable (or re-raises whatever exception might have been thrown). However, when the future is not done, the behavior of the result method changes:
- In a `concurrency.futures.Future` instance, invoking f.result() will block the caller’s thread until the result is ready - but a `timeout` option can be passed, which will through `TimeoutError`.
- `asyncio.Future.result` method does not support timeout, and `await` is the preferred way to get the result of futures in asyncio - but await doesn’t work with `concurrency.futures.Future` instances